In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import classification_report

from joblib import dump
import os

In [3]:
import os
#import cv2
import pandas as pd
#import cupy as cp  # CuPy for GPU-based NumPy operations
import numpy as np
#import tensorflow as tf
import scipy
#from skimage.feature import local_binary_pattern
#from skimage.filters import gabor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Load the data,labels and groups**

In [4]:
# Memory-mapped file paths
output_dir = '/content/drive/My Drive/Fabric Detection Project/Extracted_features'
data_file = os.path.join(output_dir, 'data_file.dat')
labels_file = os.path.join(output_dir, 'labels_file.dat')
labels_encoded_file = os.path.join(output_dir,"labels_encoded.dat")
groups_file = os.path.join(output_dir, 'groups_file.dat')

# Known values
total_images = 17780  # Total original images + augmentations
feature_size = 49162   # Number of features per image

# Load the memory-mapped arrays for reading
data = np.memmap(data_file, dtype='float32', mode='r', shape=(total_images, feature_size))
labels = np.memmap(labels_file, dtype='object', mode='r', shape=(total_images,))
labels_encoded = np.memmap(labels_encoded_file, dtype='int32', mode='w+', shape=(total_images,))
groups = np.memmap(groups_file, dtype='int32', mode='r', shape=(total_images,))

In [5]:
# Checking the shapes of data,labels and groups
print(data.shape)
print(labels.shape)
print(groups.shape)

(17780, 49162)
(17780,)
(17780,)


49162 features and 17780 rows

**Label Encoding of target variable**

In [1]:
label_encoder = LabelEncoder()

labels_encoded[:] = label_encoder.fit_transform(labels.astype(str))  # Convert object labels to string before encoding

# Step 4: Flush changes to disk
labels_encoded.flush()

# Optionally delete the old labels array if you no longer need it
del labels

print("Label encoding completed and stored in new memory-mapped array.")


NameError: name 'LabelEncoder' is not defined

In [ ]:

# Step 2: Initialize label encoder and determine chunk size
label_encoder = LabelEncoder()
chunk_size = 1000  # Choose a chunk size that fits comfortably in memory (adjust if needed)

# Step 3: Process the labels in chunks
for start in range(0, len(labels), chunk_size):
    end = min(start + chunk_size, len(labels))

    # Read a chunk of labels
    label_chunk = labels[start:end].astype(str)  # Convert to string

    # Encode the chunk
    encoded_chunk = label_encoder.fit_transform(label_chunk)

    # Write the encoded chunk to the new memory-mapped array
    labels_encoded[start:end] = encoded_chunk

    # Optionally flush after each chunk to ensure data is written to disk
    labels_encoded.flush()

# Step 4: Final flush and cleanup
labels_encoded.flush()
del labels

print("Label encoding completed and stored in new memory-mapped array in chunks.")
